In [1]:
# API KEY Loading
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
from langchain_teddynote import logging

logging.langsmith("CH06-Memory")

LangSmith 추적을 시작합니다.
[프로젝트명]
CH06-Memory


# ConversationTokenBufferMemory

1. 개요
    - 대화 기록을 관리하는 메모리 클래스 중 하나로, 대화를 토큰수 기준으로 관리
    - 지정된 토큰수를 초과하면 가장 오래된 기록을 삭제
    - LLM의 context length 정책을 고려한 메모리 관리 방식 

2. 주요 특징 
    1. Context Length 최적화:
        - LLM의 Max Context Length 내에서 효율적으로 대화 기록 유지 
        - 긴 대화에서도 최신 문맥을 유지하며 모델 호출 성능 보장 
    2. 동적 메모리 사용:
        - 대화 길이에 따라 유동적으로 기록 유지
        - 필요한 문맥을 최신 상태로 제공 
        - 대화 기록의 토큰수를 계산하고 유지하는데 약간의 추가 리소스 필요

[Reference] https://python.langchain.com/v0.1/docs/modules/memory/types/token_buffer/

In [3]:
from langchain_openai import ChatOpenAI
from langchain.memory import ConversationTokenBufferMemory

In [4]:
# 모델 정의 
model = ChatOpenAI(model_name='gpt-4o')

In [5]:
# 메모리 설정/초기화 
memory = ConversationTokenBufferMemory(
    llm=model,
    max_token_limit=200,
    return_messages=True
)

/var/folders/37/2rwxb6k520l4tqxzpx1nh8lh0000gn/T/ipykernel_26396/2854290316.py:2: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationTokenBufferMemory(


In [6]:
# 대화 내용 저장 (토큰 400개) 
memory.save_context(
    inputs={
        "human": "안녕하세요, 은행 계좌를 재발급하고 싶습니다. 어떻게 시작해야 하나요?"
    },
    outputs={
        "ai": "안녕하세요! 계좌 재발급 도와드리겠습니다. 먼저, 본인 인증을 위해 신분증을 준비해 주시겠어요? 신분증에는 주민등록증, 운전면허증, 모바일건강보험증 등이 포함됩니다."
    },
)

memory.save_context(
    inputs={
        "human": "죄송한데 운전면허증은 없고, 모바일건강보험증은 있는데 가능한가요?"
    },
    outputs={
        "ai": "네, 물론입니다! 모바일건강보험증도 본인 인증용으로 사용할 수 있습니다. 지금 화면에 보이시는 바코드를 촬영해 주시면 본인 인증 절차를 시작하겠습니다. 바코드를 스캔하면 시스템이 자동으로 인증을 진행합니다."
    },
)

memory.save_context(
    inputs={
        "human": "네, 바코드 통해 실행하여 확인 버튼 눌렀습니다."
    },
    outputs={
        "ai": "감사합니다! 본인 인증이 정상적으로 확인되었습니다. 다음 단계로 휴대폰 인증을 진행하셔야 합니다. 잠시 후 문자로 인증번호가 발송될 예정입니다. 해당 인증번호를 입력해 주세요."
    },
)

memory.save_context(
    inputs={
        "human": "인증 번호 입력하여 완료 처리하였습니다."
    },
    outputs={
        "ai": "확인했습니다! 본인 인증이 모두 완료되었습니다. 이제 계좌 재발급을 위해 새 계좌를 개설할지 기존 계좌를 그대로 재발급할지를 선택해 주세요. 또한, 통장 발행 여부와 카드 추가 발급이 필요한지도 알려주시면 됩니다."
    },
)

memory.save_context(
    inputs={
        "human": "기존 계좌를 그대로 재발급하고 싶습니다. 통장은 발행하지 않아도 되고, 카드는 추가 발급이 필요합니다."
    },
    outputs={
        "ai": "알겠습니다! 기존 계좌를 그대로 재발급하고, 카드 추가 발급을 진행하겠습니다. 최종 확인을 위해 발급받으실 카드 종류(체크카드, 신용카드 등)를 말씀해 주시면 빠르게 처리해 드리겠습니다."
    },
)

memory.save_context(
    inputs={
        "human": "체크카드로 발급받고 싶습니다."
    },
    outputs={
        "ai": "네, 체크카드로 발급 요청 확인되었습니다. 요청하신 카드는 5~7 영업일 이내에 등록된 주소로 발송될 예정입니다. 추가로 도움이 필요하시면 언제든 말씀해 주세요!"
    },
)


In [7]:
# 대화 내용 확인
memory.load_memory_variables({})["history"]

[HumanMessage(content='기존 계좌를 그대로 재발급하고 싶습니다. 통장은 발행하지 않아도 되고, 카드는 추가 발급이 필요합니다.', additional_kwargs={}, response_metadata={}),
 AIMessage(content='알겠습니다! 기존 계좌를 그대로 재발급하고, 카드 추가 발급을 진행하겠습니다. 최종 확인을 위해 발급받으실 카드 종류(체크카드, 신용카드 등)를 말씀해 주시면 빠르게 처리해 드리겠습니다.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='체크카드로 발급받고 싶습니다.', additional_kwargs={}, response_metadata={}),
 AIMessage(content='네, 체크카드로 발급 요청 확인되었습니다. 요청하신 카드는 5~7 영업일 이내에 등록된 주소로 발송될 예정입니다. 추가로 도움이 필요하시면 언제든 말씀해 주세요!', additional_kwargs={}, response_metadata={})]

-----
** End of Documents **